In [ ]:
# a shallow network with batch normalization and different data augmentation parameters

In [1]:
# import the required packages
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint

In [2]:
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

In [3]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [4]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

In [5]:
trainX, trainY, testX, testY = load_dataset()

trainX, testX = prep_pixels(trainX, testX)

170500096/170498071 [==============================] - 3s 0us/step


In [6]:
def define_model():
  model = Sequential()
  model.add(Conv2D(32, (3,3), activation = 'relu', kernel_initializer = 'he_uniform', padding='same', input_shape=(32,32,3)))
  model.add(Conv2D(32, (3,3), activation = 'relu', kernel_initializer = 'he_uniform', padding='same'))
  model.add(MaxPooling2D((2,2)))
  model.add(Flatten())
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(BatchNormalization())
  model.add(Dense(10, activation='softmax'))
  #compile model
  opt = SGD(lr=0.001,momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [12]:
model = define_model()

In [9]:
# we'll be saving the model after every epoch if it's the best so far

checkpoint = ModelCheckpoint("best_model_VGG1_augment.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='max', save_freq='epoch')

In [13]:
# create data generator
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(trainX, trainY, batch_size=64)
	
# fit model
from numpy.random import seed
seed(123)

steps = int(trainX.shape[0] / 64)
history = model.fit(it_train, steps_per_epoch=steps, epochs=400, validation_data=(testX, testY), verbose=1, callbacks = [checkpoint])


Epoch 1/400
781/781 [==============================] - 24s 31ms/step - loss: 1.8229 - accuracy: 0.3529 - val_loss: 1.4734 - val_accuracy: 0.4903

Epoch 00001: val_accuracy improved from 0.48030 to 0.49030, saving model to best_model_VGG1_augment.hdf5
Epoch 2/400
781/781 [==============================] - 24s 30ms/step - loss: 1.3739 - accuracy: 0.5105 - val_loss: 1.3301 - val_accuracy: 0.5417

Epoch 00002: val_accuracy improved from 0.49030 to 0.54170, saving model to best_model_VGG1_augment.hdf5
Epoch 3/400
781/781 [==============================] - 24s 30ms/step - loss: 1.2517 - accuracy: 0.5579 - val_loss: 1.2050 - val_accuracy: 0.5709

Epoch 00003: val_accuracy improved from 0.54170 to 0.57090, saving model to best_model_VGG1_augment.hdf5
Epoch 4/400
781/781 [==============================] - 24s 30ms/step - loss: 1.1682 - accuracy: 0.5852 - val_loss: 1.3047 - val_accuracy: 0.5349

Epoch 00004: val_accuracy did not improve from 0.57090
Epoch 5/400
781/781 [=========================

KeyboardInterrupt: ignored

In [ ]:
# for (0.1, 0.1): 0.79320 on test set

In [ ]:
## different augmentation (0.2, 0.2)

In [14]:
# build and compile model
model2 = define_model()
# create data generator
datagen = ImageDataGenerator(width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(trainX, trainY, batch_size=64)

checkpoint = ModelCheckpoint("best_model_VGG1_augment2.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='max', save_freq='epoch')

# fit model
from numpy.random import seed
seed(123)

steps = int(trainX.shape[0] / 64)
history = model2.fit(it_train, steps_per_epoch=steps, epochs=400, validation_data=(testX, testY), verbose=1, callbacks = [checkpoint])


Epoch 1/400
781/781 [==============================] - 25s 31ms/step - loss: 1.9665 - accuracy: 0.2976 - val_loss: 1.5975 - val_accuracy: 0.4192

Epoch 00001: val_accuracy improved from -inf to 0.41920, saving model to best_model_VGG1_augment2.hdf5
Epoch 2/400
781/781 [==============================] - 24s 30ms/step - loss: 1.5665 - accuracy: 0.4357 - val_loss: 1.7470 - val_accuracy: 0.4119

Epoch 00002: val_accuracy did not improve from 0.41920
Epoch 3/400
781/781 [==============================] - 24s 31ms/step - loss: 1.4743 - accuracy: 0.4707 - val_loss: 1.3948 - val_accuracy: 0.5030

Epoch 00003: val_accuracy improved from 0.41920 to 0.50300, saving model to best_model_VGG1_augment2.hdf5
Epoch 4/400
781/781 [==============================] - 24s 31ms/step - loss: 1.3939 - accuracy: 0.5039 - val_loss: 1.4375 - val_accuracy: 0.4992

Epoch 00004: val_accuracy did not improve from 0.50300
Epoch 5/400
781/781 [==============================] - 24s 30ms/step - loss: 1.3415 - accuracy: 0

In [15]:
# build and compile model
model3 = define_model()
# create data generator
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.2, horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(trainX, trainY, batch_size=64)

checkpoint = ModelCheckpoint("best_model_VGG1_augment2.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='max', save_freq='epoch')

# fit model
from numpy.random import seed
seed(123)

steps = int(trainX.shape[0] / 64)
history = model3.fit(it_train, steps_per_epoch=steps, epochs=400, validation_data=(testX, testY), verbose=1, callbacks = [checkpoint])


Epoch 1/400
781/781 [==============================] - 24s 30ms/step - loss: 1.8981 - accuracy: 0.3256 - val_loss: 1.4535 - val_accuracy: 0.4831

Epoch 00001: val_accuracy improved from -inf to 0.48310, saving model to best_model_VGG1_augment2.hdf5
Epoch 2/400
781/781 [==============================] - 23s 30ms/step - loss: 1.4625 - accuracy: 0.4785 - val_loss: 1.4217 - val_accuracy: 0.4966

Epoch 00002: val_accuracy improved from 0.48310 to 0.49660, saving model to best_model_VGG1_augment2.hdf5
Epoch 3/400
781/781 [==============================] - 23s 30ms/step - loss: 1.3411 - accuracy: 0.5226 - val_loss: 1.2316 - val_accuracy: 0.5556

Epoch 00003: val_accuracy improved from 0.49660 to 0.55560, saving model to best_model_VGG1_augment2.hdf5
Epoch 4/400
781/781 [==============================] - 23s 30ms/step - loss: 1.2558 - accuracy: 0.5526 - val_loss: 1.2097 - val_accuracy: 0.5769

Epoch 00004: val_accuracy improved from 0.55560 to 0.57690, saving model to best_model_VGG1_augment2.

In [16]:
# build and compile model
model4 = define_model()
# create data generator
datagen = ImageDataGenerator(width_shift_range=0.05, height_shift_range=0.05, horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(trainX, trainY, batch_size=64)

checkpoint = ModelCheckpoint("best_model_VGG1_augment3.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='max', save_freq='epoch')

# fit model
from numpy.random import seed
seed(123)

steps = int(trainX.shape[0] / 64)
history = model4.fit(it_train, steps_per_epoch=steps, epochs=400, validation_data=(testX, testY), verbose=1, callbacks = [checkpoint])


Epoch 1/400
781/781 [==============================] - 24s 30ms/step - loss: 1.7884 - accuracy: 0.3674 - val_loss: 1.4396 - val_accuracy: 0.4971

Epoch 00001: val_accuracy improved from -inf to 0.49710, saving model to best_model_VGG1_augment3.hdf5
Epoch 2/400
781/781 [==============================] - 23s 29ms/step - loss: 1.3008 - accuracy: 0.5419 - val_loss: 1.4570 - val_accuracy: 0.5293

Epoch 00002: val_accuracy improved from 0.49710 to 0.52930, saving model to best_model_VGG1_augment3.hdf5
Epoch 3/400
781/781 [==============================] - 23s 29ms/step - loss: 1.1625 - accuracy: 0.5937 - val_loss: 1.2851 - val_accuracy: 0.5663

Epoch 00003: val_accuracy improved from 0.52930 to 0.56630, saving model to best_model_VGG1_augment3.hdf5
Epoch 4/400
781/781 [==============================] - 23s 29ms/step - loss: 1.0923 - accuracy: 0.6187 - val_loss: 1.1725 - val_accuracy: 0.5965

Epoch 00004: val_accuracy improved from 0.56630 to 0.59650, saving model to best_model_VGG1_augment3.

KeyboardInterrupt: ignored

In [ ]:
# ten model sie ewidentnie przetreonwuje

In [7]:
# build and compile model
model5 = define_model()
# create data generator
datagen = ImageDataGenerator(width_shift_range=0.01, height_shift_range=0.01, featurewise_center=True, featurewise_std_normalization=True,
                             horizontal_flip=True, rotation_range=20)
# prepare iterator
it_train = datagen.flow(trainX, trainY, batch_size=64)

checkpoint = ModelCheckpoint("best_model_VGG1_augment4.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='max', save_freq='epoch')

# fit model
from numpy.random import seed
seed(123)

steps = int(trainX.shape[0] / 64)
history = model5.fit(it_train, steps_per_epoch=steps, epochs=400, validation_data=(testX, testY), verbose=1, callbacks = [checkpoint])


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/400
781/781 [==============================] - 58s 33ms/step - loss: 1.7673 - accuracy: 0.3758 - val_loss: 1.4717 - val_accuracy: 0.4955

Epoch 00001: val_accuracy improved from -inf to 0.49550, saving model to best_model_VGG1_augment4.hdf5
Epoch 2/400
781/781 [==============================] - 25s 32ms/step - loss: 1.2991 - accuracy: 0.5408 - val_loss: 1.4042 - val_accuracy: 0.5233

Epoch 00002: val_accuracy improved from 0.49550 to 0.52330, saving model to best_model_VGG1_augment4.hdf5
Epoch 3/400
781/781 [==============================] - 25s 32ms/step - loss: 1.1548 - accuracy: 0.5940 - val_loss: 1.1512 - val_accuracy: 0.5991

Epoch 00003: val_accuracy improved from 0.52330 to 0.59910, saving model to best_model_VGG1_augment4.hdf5
Epoch 4/400
781/781 [==============================] - 25s 32ms/step - loss: 1.0947 - accuracy: 0.6146 - val_loss: 1.2189 - val_accuracy: 0.5900

Epoch 00004: val_accuracy did not improve from 0.59910
Epoch 5/400
781/781 [=========================

KeyboardInterrupt: ignored